In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = (SparkSession
         .builder
         .appName("Analyzing the vocabulary of Pride and Prejudice.")
         .getOrCreate())

24/02/01 19:10:26 WARN Utils: Your hostname, Jareds-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.122.39.178 instead (on interface en0)
24/02/01 19:10:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/01 19:10:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


24/02/01 19:10:38 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [3]:
spark.sparkContext

<SparkContext master=local[*] appName=Analyzing the vocabulary of Pride and Prejudice.>

In [5]:
# configuring the log level 
# aka configuring how chatty spark gets

#   pyspark shell defaults to WARN which could be quite chatty
#   scripts using PySpark defaults to INFO which can be considered oversharing


spark.sparkContext.setLogLevel("WARN")

In [6]:
spark.read  # display a DataFrameReader object
# DataFrameReader objects read data into a data frame

## Step 1: Ingesting / Extracting Data

In [8]:
#   Reading in the Jane Austen novel
#   The novel is stored in a .txt file in ./data/

book = spark.read.text("./data/1342-0.txt")

In [9]:
book    # notice this doesn't output any data to the screen

        # instead it prints the schema (names of columns and their type)

DataFrame[value: string]

In PySpark, each column has a type: it represents how the value is represented by Spark's engine.

Knowing the type attached to each column -> helpful in seing what you can and cannot do with the data in it.

In the above example, we have ```one``` column named ```value``` composed of a ```string```

In [10]:
book.printSchema()

root
 |-- value: string (nullable = true)



In [11]:
print(book.dtypes)

[('value', 'string')]


In [17]:
book.show(10, truncate = 50)

+--------------------------------------------------+
|                                             value|
+--------------------------------------------------+
|The Project Gutenberg EBook of Pride and Prejud...|
|                                                  |
|This eBook is for the use of anyone anywhere at...|
|almost no restrictions whatsoever.  You may cop...|
|re-use it under the terms of the Project Gutenb...|
|    with this eBook or online at www.gutenberg.org|
|                                                  |
|                                                  |
|                        Title: Pride and Prejudice|
|                                                  |
+--------------------------------------------------+
only showing top 10 rows



## Step 2: Transformations (Simple): Tokenize a Sentence to Words

In [22]:
from pyspark.sql.functions import split 

lines = book.select(split(book.value, " ").alias("line"))
lines.show(5)

# seems very much like writing SQL
# SELECT split(book.value, " ") as "line" FROM book LIMIT 5;

+--------------------+
|                line|
+--------------------+
|[The, Project, Gu...|
|                  []|
|[This, eBook, is,...|
|[almost, no, rest...|
|[re-use, it, unde...|
+--------------------+
only showing top 5 rows



In [28]:
lines

DataFrame[line: array<string>]

In [29]:
lines.printSchema()

root
 |-- line: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [30]:
from pyspark.sql.functions import explode, col

In [31]:
words = lines.select(explode(col("line")).alias("word"))

In [32]:
words.show(15)

+----------+
|      word|
+----------+
|       The|
|   Project|
| Gutenberg|
|     EBook|
|        of|
|     Pride|
|       and|
|Prejudice,|
|        by|
|      Jane|
|    Austen|
|          |
|      This|
|     eBook|
|        is|
+----------+
only showing top 15 rows



In [33]:
from pyspark.sql.functions import lower

words_lower = words.select(lower(col("word")).alias("word_lower"))

words_lower.show()

+----------+
|word_lower|
+----------+
|       the|
|   project|
| gutenberg|
|     ebook|
|        of|
|     pride|
|       and|
|prejudice,|
|        by|
|      jane|
|    austen|
|          |
|      this|
|     ebook|
|        is|
|       for|
|       the|
|       use|
|        of|
|    anyone|
+----------+
only showing top 20 rows

